In [88]:
import librosa
import soundfile
import os
import glob
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [89]:
#DataFlair - Extract features (mfcc, chroma, mel) from a sound file

def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, chroma))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result

In [90]:
# DataFlair - Emotions in the RAVDESS dataset
emotions = {
    '01': 'neutral',
    '02': 'calm',
    '03': 'happy',
    '04': 'sad',
    '05': 'angry',
    '06': 'fearful',
    '07': 'disgust',
    '08': 'surprised'
}

#DataFlair - Emotions to observe
observed_emotions = ['calm', 'happy', 'fearful', 'disgust', 'angry']

In [91]:
# DataFlair - Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x, y = [], []
    for file in glob.glob('./ravdess_data/Actor_*/*.wav'):

        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [92]:
# DataFlair - Split the dataset
x_train, x_test, y_train, y_test = load_data(test_size=0.25)

In [80]:
# DataFlair - Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(624, 336)


In [81]:
# DataFlair - Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 141


In [82]:
# DataFlair - Initialize the Multi Layer Perceptron Classifier
model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=1000)

In [83]:
# DataFlair - Train the model
model.fit(x_train, y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=1000)

In [87]:
# DataFlair - Predict for the test set
y_pred = model.predict(x_test)
print(x_test)
print(y_pred)

[[1.00000000e+00 5.98885536e-01 6.68151736e-01 ... 9.17843194e-04
  5.20892907e-04 4.23721154e-04]
 [1.00000000e+00 5.64036846e-01 5.23739696e-01 ... 5.45278145e-03
  3.56062246e-03 2.32826290e-03]
 [1.00000000e+00 4.38117981e-01 3.32518280e-01 ... 3.65081453e-03
  1.93907868e-03 1.06057874e-03]
 ...
 [1.00000000e+00 5.95862925e-01 6.31316304e-01 ... 5.45494504e-06
  4.09651238e-06 1.95349662e-06]
 [1.00000000e+00 6.09680712e-01 6.27815664e-01 ... 6.37499352e-06
  4.62216485e-06 3.34434867e-06]
 [1.00000000e+00 5.18337131e-01 5.81094146e-01 ... 4.67928359e-03
  2.99159461e-03 1.92126387e-03]]
['disgust' 'angry' 'happy' 'fearful' 'disgust' 'calm' 'disgust' 'angry'
 'fearful' 'disgust' 'disgust' 'happy' 'disgust' 'angry' 'disgust' 'happy'
 'happy' 'disgust' 'happy' 'calm' 'calm' 'disgust' 'happy' 'disgust'
 'happy' 'happy' 'calm' 'fearful' 'calm' 'angry' 'fearful' 'disgust'
 'angry' 'angry' 'fearful' 'disgust' 'disgust' 'fearful' 'happy' 'disgust'
 'fearful' 'calm' 'angry' 'fearful' 'dis

In [85]:
# DataFlair - Calculate the accuracy of our model
accuracy = accuracy_score(y_true=y_test, y_pred=y_pred)

# DataFlair - Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 54.17%
